<a href="https://colab.research.google.com/github/vksyr/IST718/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd  # data frame operations
import numpy as np  # arrays and math functions
from scipy.stats import uniform  # for training-and-test split
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
import matplotlib.pyplot as plt  # 2D plotting
import seaborn as sns  # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING
import math

import pandas as pd


In [0]:
# Dataset was converted to CSV with this script: 
# https://pjreddie.com/projects/mnist-in-csv/
def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

convert("/content/drive/My Drive/IST718/HW3/train-images-idx3-ubyte", "/content/drive/My Drive/IST718/HW3/train-labels-idx1-ubyte",
        "/content/drive/My Drive/IST718/HW3/mnist_train.csv", 60000)
convert("/content/drive/My Drive/IST718/HW3/t10k-images-idx3-ubyte", "/content/drive/My Drive/IST718/HW3/t10k-labels-idx1-ubyte",
        "/content/drive/My Drive/IST718/HW3/mnist_test.csv", 10000)

In [0]:
# create pandas df
cols = np.array(['label'])
for i in range(1,785):
  cols = np.append(cols, ['pixel' + str(i)])

train_images = pd.read_csv('/content/drive/My Drive/IST718/HW3/mnist_train.csv', 
                          header=None, names=cols)
test_images = pd.read_csv('/content/drive/My Drive/IST718/HW3/mnist_test.csv', 
                          header=None, names=cols)

In [0]:
'''
# plotting an image
import matplotlib.pyplot as plt
image = np.asarray(train_images.loc[0][1:]).squeeze()
plt.imshow(image)
plt.show()


# plotting an image
import matplotlib.pyplot as plt
image = np.asarray(data[0]).squeeze()
plt.imshow(image)
plt.show()
'''

Tensorflow

In [4]:
%tensorflow_version 1.x  #gpu
import tensorflow as tf
tf.compat.v1.disable_eager_execution() 
# needs to be disabled for placeholder
# enabled for iteration

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  #gpu`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [0]:
'''
LABEL_COLUMN = 'label'
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      column_names = cols, 
      column_defaults = [0] * 785, 
      **kwargs)
  return dataset

raw_train_data = get_dataset('/content/drive/My Drive/IST718/HW3/mnist_train.csv')
raw_test_data = get_dataset('/content/drive/My Drive/IST718/HW3/mnist_test.csv')
'''

In [0]:
'''
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

#show_batch(raw_train_data)

# pack the data into a vector before passing it off to the model
# from https://www.tensorflow.org/tutorials/load_data/csv
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels


packed_train_data = raw_train_data.map(
    PackNumericFeatures(cols[1:]))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(cols[1:]))

#example_batch, labels_batch = next(iter(packed_train_data)) 
'''

In [0]:
'''
import numpy as np

def next_batch(num, data, labels):

#    Return a total of `num` random samples and labels. 

    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


next_batch(batch_size, packed_train_data.batch(), packed_train_data)
'''

In [5]:
from tensorflow.examples.tutorials.mnist import input_data

# Read data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
image_size = 28
labels_size = 10
learning_rate = 0.05
steps_number = 1000
batch_size = 100

# Define placeholders
training_data = tf.placeholder(tf.float32, [None, image_size*image_size])
labels = tf.placeholder(tf.float32, [None, labels_size])

# Variables to be tuned
W = tf.Variable(tf.truncated_normal([image_size*image_size, labels_size], stddev=0.1))
b = tf.Variable(tf.constant(0.1, shape=[labels_size]))

# Build the network (only output layer)
output = tf.matmul(training_data, W) + b

# Define the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=output))

# Training step
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Accuracy calculation
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# Run the training
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(steps_number):
  # Get the next batch
  input_batch, labels_batch = mnist.train.next_batch(batch_size)
  feed_dict = {training_data: input_batch, labels: labels_batch}

  # Run the training step
  train_step.run(feed_dict=feed_dict)

  # Print the accuracy progress on the batch every 100 steps
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict=feed_dict)
    print("Step %d, training batch accuracy %g %%"%(i, train_accuracy*100))

In [9]:
# Evaluate on the test set
test_accuracy = accuracy.eval(feed_dict={training_data: mnist.test.images, labels: mnist.test.labels})
print("Test accuracy: %g %%"%(test_accuracy*100))

Test accuracy: 89.83 %


In [7]:
'''
# MODEL
# CREATE PLACEHOLDER VARIABLES FOR OPERATION MANIPULATION
# THE 784 MATCHES THE VECTOR SIZE OF THE MNIST IMAGES - 28*28
x = tf.placeholder(tf.float32, [None, 784])

# MODEL
# CREATE WEIGHTS & BIASES VARIABLES
# IN TF, OUR MODEL PARAMETERS ARE OFTEN MANAGED AS VARIABLES
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# MODEL
# CREATE MODEL - USES SOFTMAX AS THE ACTIVATION FUNCTION
# REMEMBER GOAL FOR ACTIVATION FUNCTION IS TO "SHAPE" THE 
# OUTPUT INTO A PROBABILITY DISTRO OVER THE 10 CLASSES
y = tf.nn.softmax(tf.matmul(x, W) + b)

# MODEL
# CREATE PREDICTED VARIABLE Y-HAT
# AND USE CROSS ENTROPY TO DETERMINE LOSS
# CROSS ENTROPY - HOW INEFFICIENT ARE OUR PREDICTIONS?
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# MODEL
# TRAIN USING GRADIENT DESCENT
# LEARNING RATE AT MIDPOINT - .5 - MAKE SMALL ADJUSTMENTS TO MINIMIZE COST
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

'''

AttributeError: ignored